## Introduction:

This notebook implements the LeNet5 neural network architecture using Theano to calssify MNIST dataset.  

LeNet5 is a convolutional neural network, good for classifying images. 

CNN consists of thre main phases:
1. Convolutional layer
2. RELU
3. Pooling (used for dimensionality reduction)

==> Finally, the fully conntected layer has the ability to connect the whole results to be able to classify the whole image

One of the main disadvantages of CNN is it belongs to unsupervised learning approaches hence it requires huge number of images to be well trained .. 

### Model assumptions:

 - LeNetConvPool doesn't implement location-specific gain and bias parameters
 - LeNetConvPool doesn't implement pooling by average, it implements pooling
   by max.
 - Digit classification is implemented with a logistic regression rather than
   an RBF network
 - LeNet5 was not fully-connected convolutions at second layer
 
As a prerequisite you are expected to classify MNIST dataset using Logistic regresion (utilized in layer3 - to generate the output) and MLP as we gonna utilize these previous implementations .. 

The topology of the constructed network is as follows (one input layer, single hidden layer and one output layer) 

In [8]:
import os
import sys
import timeit

import numpy as np

import theano
import theano.tensor as T
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d

from logistic_sgd import LogisticRegression, load_data
from mlp import HiddenLayer

import warnings 
warnings.filterwarnings("ignore")

Max pooling will be implemented in this notebook .. 
in general the pooling step is to select the optimal (minimum) number of features from the whole number of input features .. 

In [9]:
## Pool step .. 

class LeNetConvPoolLayer(object):
    def __init__(self, rng, input, filter_shape, image_shape, poolsize=(2, 2)):
        ## filter_shape: (number of filters, num input feature maps,filter height, filter width)
        ## image_shape: (batch size, num input feature maps, image height, image width)
        assert image_shape[1] == filter_shape[1]  ## assert statement helps you find bugs more quickly and with less pain. 
        self.input = input
        
        # inputs to each hidden unit ==> "num input feature maps * filter height * filter width"
        fan_in = np.prod(filter_shape[1:])
        
        # each unit in the lower layer receives a gradient from:
        # "num output feature maps * filter height * filter width" / pooling size
        
        fan_out = (filter_shape[0] * np.prod(filter_shape[2:]) // np.prod(poolsize))
        
        # initialize weights with random weights .. tanh activation function is utilized in generating the weights
        W_bound = np.sqrt(6. / (fan_in + fan_out))
        self.W = theano.shared(
            np.asarray(rng.uniform(low=-W_bound, high=W_bound, size=filter_shape),
                dtype=theano.config.floatX),borrow=True)
        
        
         # the bias is a 1D tensor -- one bias per output feature map
        b_values = np.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

         # convolve input feature maps with filters
        conv_out = conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            input_shape=image_shape)
        
        
         # pool each feature map individually, using maxpooling
        pooled_out = pool.pool_2d(
            input=conv_out,
            ds=poolsize,
            ignore_border=True)
        
         # pool each feature map individually, using maxpooling
        pooled_out = pool.pool_2d(   ## pool from theano lib
            input=conv_out,
            ds=poolsize,
            ignore_border=True
        )
        
         # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map width & height
        self.output = T.tanh(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

The lower-layers are composed to alternating convolution and max-pooling layers. The upper-layers however are fully-connected and correspond to a traditional MLP (hidden layer + logistic regression)..

From an implementation point of view, this means lower-layers operate on 4D tensors. These are then
flattened to a 2D matrix of rasterized feature maps, to be compatible with our previous MLP implementation ..

In [10]:
def evaluate_lenet5(learning_rate=0.1, n_epochs=200, dataset='mnist.pkl.gz', nkerns=[20, 50], batch_size=500):
    rng = np.random.RandomState(23455)
    datasets = load_data(dataset)
    
    train_set_x, train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    
    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0]
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0]
    n_test_batches = test_set_x.get_value(borrow=True).shape[0]
    n_train_batches //= batch_size
    n_valid_batches //= batch_size
    n_test_batches //= batch_size
    
     ###############
    # MODEL BUILDING #
    ###############
    
    
     # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch

    x = T.matrix('x')   # the data is presented as rasterized images
    y = T.ivector('y')  # the labels are presented as 1D vector of [int] labels
    
    
    # Reshape matrix of rasterized images of shape (batch_size, 28 * 28)
    # to a 4D tensor, compatible with our LeNetConvPoolLayer
    # (28, 28) is the size of MNIST images.
    layer0_input = x.reshape((batch_size, 1, 28, 28))
    
     # Construct the first convolutional pooling layer:
    # filtering reduces the image size to (28-5+1 , 28-5+1) = (24, 24)
    # maxpooling reduces this further to (24/2, 24/2) = (12, 12)
    # 4D output tensor is thus of shape (batch_size, nkerns[0], 12, 12)
    layer0 = LeNetConvPoolLayer(
        rng,
        input=layer0_input,
        image_shape=(batch_size, 1, 28, 28),
        filter_shape=(nkerns[0], 1, 5, 5),
        poolsize=(2, 2))

    
    # Construct the second convolutional pooling layer
    # filtering reduces the image size to (12-5+1, 12-5+1) = (8, 8)
    # maxpooling reduces this further to (8/2, 8/2) = (4, 4)
    # 4D output tensor is thus of shape (batch_size, nkerns[1], 4, 4)
    layer1 = LeNetConvPoolLayer(
        rng,
        input=layer0.output,
        image_shape=(batch_size, nkerns[0], 12, 12),
        filter_shape=(nkerns[1], nkerns[0], 5, 5),
        poolsize=(2, 2))
    
    # the HiddenLayer being fully-connected, it operates on 2D matrices of
    # shape (batch_size, num_pixels) (i.e matrix of rasterized images).
    # This will generate a matrix of shape (batch_size, nkerns[1] * 4 * 4),
    # or (500, 50 * 4 * 4) = (500, 800) with the default values.
    layer2_input = layer1.output.flatten(2)

      # construct a fully-connected sigmoidal layer
    layer2 = HiddenLayer(
        rng,
        input=layer2_input,
        n_in=nkerns[1] * 4 * 4,
        n_out=500,
        activation=T.tanh)
    
    # classify the values of the fully-connected sigmoidal layer
    layer3 = LogisticRegression(input=layer2.output, n_in=500, n_out=10)
    
    # the cost we minimize during training is the -ve log likelihood of the model
    cost = layer3.negative_log_likelihood(y)

    
     # create a function to compute the mistakes that are made by the model
    test_model = theano.function(
        [index],
        layer3.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]})
    
    ## model validation
    validate_model = theano.function(
        [index],
        layer3.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]})
    
    ## hence gradients will be calculated in terms of ALL model parameters .. 
    # create a list of all model parameters to be fit by gradient descent
    params = layer3.params + layer2.params + layer1.params + layer0.params
    
     # create a list of gradients for all model parameters
    grads = T.grad(cost, params)
    
      # train_model is a function that updates the model parameters by
    # SGD Since this model has many parameters, it would be tedious to
    # manually create an update rule for each model parameter. We thus
    # create the updates list by automatically looping over all
    # (params[i], grads[i]) pairs.
    updates = [(param_i, param_i - learning_rate * grad_i) for param_i, grad_i in zip(params, grads)]
    
    train_model = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]})
    
     ###############
    # MODEL TRAINING #
    ###############
    
    # early-stopping parameters
    patience = 10000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                           # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = min(n_train_batches, patience // 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = np.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = False
    this_validation_loss = 0
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):

            iter = (epoch - 1) * n_train_batches + minibatch_index

            if iter % 100 == 0:
                print('training @ iter = ', iter)
            cost_ij = train_model(minibatch_index)

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i in range(n_valid_batches)]
                this_validation_loss = np.mean(validation_losses)
                print('epoch %i, minibatch %i/%i, validation error %f %%' %
                      (epoch, minibatch_index + 1, n_train_batches,
                       this_validation_loss * 100.))
            
             # if we got the best validation score until now
            if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss * improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                    # save best validation score and iteration number
                    best_validation_loss = this_validation_loss
                    best_iter = iter
                    
                    # test it on the test set
                    test_losses = [test_model(i) for i in range(n_test_batches)]
                    test_score = np.mean(test_losses)
                    print(('     epoch %i, minibatch %i/%i, test error of '
                           'best model %f %%') %
                          (epoch, minibatch_index + 1, n_train_batches, test_score * 100.))
            
            ## stopping condition
            if patience <= iter:
                done_looping = True
                break
            
    end_time = timeit.default_timer()
    print('Best validation score of %f %% obtained at iteration %i, '
          'with test performance %f %%' % (best_validation_loss * 100., best_iter + 1, test_score * 100.))

In [11]:
if __name__ == '__main__':
    evaluate_lenet5()

... loading data
('training @ iter = ', 0)
     epoch 1, minibatch 1/100, test error of best model 61.990000 %
epoch 1, minibatch 100/100, validation error 9.230000 %
('training @ iter = ', 100)
epoch 2, minibatch 100/100, validation error 6.180000 %
('training @ iter = ', 200)
epoch 3, minibatch 100/100, validation error 4.640000 %
('training @ iter = ', 300)
epoch 4, minibatch 100/100, validation error 3.500000 %
('training @ iter = ', 400)
epoch 5, minibatch 100/100, validation error 3.020000 %
('training @ iter = ', 500)
epoch 6, minibatch 100/100, validation error 2.780000 %
('training @ iter = ', 600)
epoch 7, minibatch 100/100, validation error 2.480000 %
('training @ iter = ', 700)
epoch 8, minibatch 100/100, validation error 2.290000 %
('training @ iter = ', 800)
epoch 9, minibatch 100/100, validation error 2.160000 %
('training @ iter = ', 900)
epoch 10, minibatch 100/100, validation error 1.970000 %
('training @ iter = ', 1000)
epoch 11, minibatch 100/100, validation error 1

epoch 96, minibatch 100/100, validation error 0.940000 %
('training @ iter = ', 9600)
epoch 97, minibatch 100/100, validation error 0.930000 %
('training @ iter = ', 9700)
epoch 98, minibatch 100/100, validation error 0.930000 %
('training @ iter = ', 9800)
epoch 99, minibatch 100/100, validation error 0.940000 %
('training @ iter = ', 9900)
epoch 100, minibatch 100/100, validation error 0.940000 %
('training @ iter = ', 10000)
Best validation score of 0.000000 % obtained at iteration 1, with test performance 61.990000 %


## Comment:
The running time of CNN is even higher than MLP and simple logistic regression .. 
this is due to the large number of parameters (although shared variables concept is there) and the complexity of the network topology .. 

The number of filters used in CNNs is typically much smaller than the number of hidden units in MLPs
and depends on the size of the feature maps (itself a function of input image size and filter shapes).

The test error deceases rapidely from 61.9% in the first 100 itrs to only 9.23% in the 2nd 100 itrs .. and cont. decreasing (but with lower rate) up to about 0.94%